project 1

In [12]:
import pandas as pd
import pickle


In [9]:
class Preprocessor():
    @staticmethod
    def process(doc_query):
       res = Preprocessor.tokenizeDocument(doc_query)
       return res

    @staticmethod
    def tokenizeDocument(sentence):
        return sentence.split(' ')

In [6]:


class IndexModel:

    def __init__(self, documents_df,index_file=None ,meta_file=None):
        
        self._index = {}
        self.create_new_index(documents_df)
        
        if index_file==None and meta_file==None :
           self.create_new_index
           
        else :
            self.read_index    
        

    def create_new_index(self, documents_df):
        
        termdoc = documents_df.to_dict('list')
        unique_terms = set()
        doc_ids = termdoc['id']

       
        for terms in termdoc['ntext']:
            unique_terms.update(terms)
        
        term_ids = sorted(list(unique_terms))  
        term_to_id = {term: idx for idx, term in enumerate(term_ids)}
        doc_to_id = {doc_id: idx for idx, doc_id in enumerate(doc_ids)}
        
        
        self._index_matrix = [[0 for _ in range(len(doc_ids))] for _ in range(len(term_ids))]
        
        
        for i, terms in enumerate(termdoc['ntext']):
            for term in terms:
                term_idx = term_to_id[term]
                doc_idx = doc_to_id[doc_ids[i]]
                self._index_matrix[term_idx][doc_idx] = 1

        self._term_to_id = term_to_id
        self._doc_to_id = doc_to_id

    def get_term_vector(self, term):  
        
        if term in self._term_to_id:
            term_idx = self._term_to_id[term]
            return self._index_matrix[term_idx]
        else:
            return [0 for _ in range(len(self._doc_to_id))]
        
    
    def read_index(self, index_file, meta_file):
        
        
        with open(index_file, 'rb') as f:
            self._index_matrix = pickle.load(f)
        
        
        with open(meta_file, 'rb') as f:
            meta_data = pickle.load(f)
        
        self._term_to_id = meta_data['term_to_id']
        self._doc_to_id = meta_data['doc_to_id']
        
          
        
    
        
                        


   

In [15]:
#ex :
documents = {
    'id': ["TheJungle", "FruitTypes", "HealthyLife"],
    'ntext': [["tree", "garden", "juice"], ["tree", "apple"], ["apple", "fruit", "juice"]]
}

documents_df = pd.DataFrame(documents)
index_model = IndexModel(documents_df)

print("TermId:", index_model._term_to_id)
print("DocId:", index_model._doc_to_id)
print("Index Matrix:")
for row in index_model._index_matrix:
    print(row)

# Testing term incidence vectors
print("Incidence vector for 'tree':", index_model.term_incidence_vector("tree"))
print("Incidence vector for 'juice':", index_model.term_incidence_vector("juice"))


TermId: {'apple': 0, 'fruit': 1, 'garden': 2, 'juice': 3, 'tree': 4}
DocId: {'TheJungle': 0, 'FruitTypes': 1, 'HealthyLife': 2}
Index Matrix:
[0, 1, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 1]
[1, 1, 0]
Incidence vector for 'tree': [1, 1, 0]
Incidence vector for 'juice': [1, 0, 1]


In [14]:
# import pandas as pd

# class IndexModel:
#     def __init__(self, documents_df=None, index_path=None, meta_path=None):
#         self.documents_df = documents_df
#         self.term_id = {}
#         self.doc_id = {}
#         self.tdim_index = []

#         if index_path and meta_path:
#             self.index_read(index_path, meta_path)
#         elif documents_df is not None:
#             self.index_new_create()

#     def index_new_create(self):
#         termdoc = self.documents_df.to_dict('list')

#         # تحديد الأرقام التعريفية للوثائق
#         self.doc_id = {doc: i for i, doc in enumerate(termdoc['id'])}

#         # جمع المصطلحات الفريدة وتحديد الأرقام التعريفية للمصطلحات
#         uniqterm = set()
#         for terms in termdoc['ntext']:
#             uniqterm.update(terms)
#         self.term_id = {term: i for i, term in enumerate(uniqterm)}

#         # إنشاء مصفوفة TDIM
#         num_terms = len(self.term_id)
#         num_docs = len(self.doc_id)
#         self.tdim_index = [[0] * num_docs for _ in range(num_terms)]

#         # تعبئة مصفوفة TDIM
#         for i, doc_terms in enumerate(termdoc['ntext']):
#             for term in doc_terms:
#                 term_index = self.term_id[term]
#                 doc_index = self.doc_id[termdoc['id'][i]]
#                 self.tdim_index[term_index][doc_index] = 1

#     def index_read(self, index_path, meta_path):
#         import pickle
        
#         # قراءة ملف الفهرس
#         with open(index_path, 'rb') as f:
#             self.tdim_index = pickle.load(f)
        
#         # قراءة ملف معلومات الفهرس
#         with open(meta_path, 'rb') as f:
#             meta_data = pickle.load(f)
#             self.term_id = meta_data[0]
#             self.doc_id = meta_data[1]

#     def vector_term_get(self, term):
#         try:
#             return self.tdim_index[self.term_id[term]]
#         except KeyError:
#             return [0] * len(self.doc_id)

#     def save_index(self, index_path, meta_path):
#         import pickle
        
#         # حفظ مصفوفة الفهرس
#         with open(index_path, 'wb') as f:
#             pickle.dump(self.tdim_index, f)
        
#         # حفظ معلومات الفهرس
#         meta_data = [self.term_id, self.doc_id]
#         with open(meta_path, 'wb') as f:
#             pickle.dump(meta_data, f)

# # مثال تطبيقي
# data = {'id': ["TheJungle", "FruitTypes", "HealthyLife"],
#         'ntext': [['tree', 'garden', 'juice'], ['apple', 'fruit'], ['apple', 'juice', 'fruit']]}

# documents_df = pd.DataFrame(data)

# # إنشاء نموذج الفهرسة
# index_model = IndexModel(documents_df=documents_df)

# # إنشاء الفهرس
# index_model.index_new_create()

# # طباعة الأرقام التعريفية للمصطلحات والوثائق
# print(f"Term Id: {index_model.term_id}")
# print(f"Doc Id: {index_model.doc_id}")

# # طباعة مصفوفة TDIM
# print("TDIM Index:")
# for row in index_model.tdim_index:
#     print(row)

# # الحصول على متجه وقوع المصطلح "apple"
# apple_vector = index_model.vector_term_get('apple')
# print(f"متجه وقوع المصطلح 'apple': {apple_vector}")

# # حفظ الفهرس ومعلوماته
# index_model.save_index('index.myindex', 'meta.ids_td')

# # قراءة الفهرس ومعلوماته من الملفات
# new_index_model = IndexModel(index_path='index.myindex', meta_path='meta.ids_td')

# # طباعة الأرقام التعريفية للمصطلحات والوثائق من الفهرس المحمل
# print(f"Loaded Term Id: {new_index_model.term_id}")
# print(f"Loaded Doc Id: {new_index_model.doc_id}")

# # طباعة مصفوفة TDIM من الفهرس المحمل
# print("Loaded TDIM Index:")
# for row in new_index_model.tdim_index:
#     print(row)
